In [1]:
import requests as rq
import json
import pandas as pd
pd.options.plotting.backend = "plotly"

In [2]:
# Get API key
def get_key():
    f = open("/home/vikas/Documents/CG_key.json")
    key_dict = json.load(f)
    return key_dict["key"] 

## Check API status

In [3]:
PUB_URL = "https://api.coingecko.com/api/v3"
PRO_URL = "https://pro-api.coingecko.com/api/v3"

In [4]:
def get_response(endpoint, headers, params, URL):
    url = "".join((URL, endpoint))
    response = rq.get(url, headers = headers, params = params)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to fetch data, check status code {response.status_code}")    

In [5]:
use_demo = {"accept": "application/json"}

In [7]:
get_response("/ping", use_demo, "", PUB_URL)

{'gecko_says': '(V3) To the Moon!'}

In [6]:
use_pro = {
         "accept": "application/json",
         "x-cg-pro-api-key" : get_key()          
}

## NFTs

#### Demo API

In [51]:
nft_params = {
            "order": "market_cap_usd_desc",
            "per_page": 10,
            "page": 1
}

In [52]:
nft_list_response = get_response("/nfts/list", use_demo, nft_params, PUB_URL)

In [53]:
df_nft = pd.DataFrame(nft_list_response)

In [54]:
df_nft

,id,contract_address,name,asset_platform_id,symbol
0,cryptopunks,0xb47e3cd837dDF8e4c57F05d70Ab865de6e193BBB,CryptoPunks,ethereum,PUNK
1,bored-ape-yacht-club,0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d,Bored Ape Yacht Club,ethereum,BAYC
2,the-crop-collective,0xe5863d4a57b1e2d7031f7969fee4f56bb04c0f17,The Crop Collective,ethereum,$TCC
3,more-loot,0x1dfe7ca09e99d10835bf73044a23b73fc20623df,More Loot,ethereum,mloot
4,mutant-ape-yacht-club,0x60e4d786628fea6478f785a6d7e704777c86a7c6,Mutant Ape Yacht Club,ethereum,MAYC
5,autoglyphs,0xd4e4078ca3495de5b1d4db434bebc5a986197782,Autoglyphs,ethereum,☵
6,chromie-squiggle-by-snowfro,0x059edd72cd353df5106d2b9cc5ab83a52287ac3a,Chromie Squiggle by Snowfro,ethereum,BLOCKS
7,the-almighty-sparrows,0xcaa1c15fe6c71033172133c65c2e6ca327c13ac2,The Almighty Sparrows,ethereum,Sparrows
8,fidenza-by-tyler-hobbs,0xa7d8d9ef8d8ce8992df33d8b8cf4aebabd5bd270,Fidenza by Tyler Hobbs,ethereum,BLOCKS
9,sandbox,0x5CC5B05a8A13E3fBDB0BB9FcCd98D38e50F90c38,LAND,ethereum,Sandbox's LANDs


In [13]:
df_nft.id[4]

'syltare-dawn-of-east-klaytn'

In [55]:
def get_nft_current_data(index, df_nft, use_demo = use_demo, PUB_URL = PUB_URL):
    nft_url = f"/nfts/{df_nft.id[index]}"
    nft_current_response = get_response(nft_url, use_demo, nft_params, PUB_URL)
    
    new_data = dict(id = nft_current_response["id"], 
                    name = nft_current_response["name"],
                    symbol = nft_current_response["symbol"],
                    floor_price_usd = nft_current_response["floor_price"]["usd"])
    
    return new_data

In [60]:
def get_nft_floor_prices(order_by, per_page):

    assert per_page <= 250, "Number of entries per page should be less than 250"

    nft_params = {
                 "order": order_by,
                 "per_page": per_page,
                 "page": 1
                 }

    nft_list_response = get_response("/nfts/list", use_demo, nft_params, PUB_URL)
    df_nft = pd.DataFrame(nft_list_response)

    nft_list_all = []

    for i in range(len(df_nft.id)):
        new_data = get_nft_current_data(i, df_nft)
        nft_list_all.append(new_data)

    df_floor_price = pd.DataFrame(nft_list_all)

    return df_floor_price   

In [61]:
df_floor_price = get_nft_floor_prices("market_cap_usd_desc", 10)

In [62]:
df_floor_price

,id,name,symbol,floor_price_usd
0,cryptopunks,CryptoPunks,PUNK,88125.00
1,bored-ape-yacht-club,Bored Ape Yacht Club,BAYC,52875.00
2,the-crop-collective,The Crop Collective,$TCC,766855.00
3,more-loot,More Loot,mloot,1654.58
4,mutant-ape-yacht-club,Mutant Ape Yacht Club,MAYC,9981.89
5,autoglyphs,Autoglyphs,☵,319591.00
6,chromie-squiggle-by-snowfro,Chromie Squiggle by Snowfro,BLOCKS,16262.42
7,the-almighty-sparrows,The Almighty Sparrows,Sparrows,144933.00
8,fidenza-by-tyler-hobbs,Fidenza by Tyler Hobbs,BLOCKS,110292.00
9,sandbox,LAND,Sandbox's LANDs,601.68


#### Pro API

In [17]:
page_params = {
            "per_page": 100,
            "page": 1,
            "order": "h24_volume_usd_desc"    
}

In [18]:
nft_markets = get_response("/nfts/markets", use_pro, page_params, PRO_URL)

In [19]:
nft_markets[10];

In [20]:
def get_nft_markets(num_entries, order_by):
    
    page_params = {
                    "per_page": num_entries,
                    "page": 1,
                    "order": order_by    
                  }
    
    nft_markets = get_response("/nfts/markets", use_pro, page_params, PRO_URL) 
    nft_all = []

    for i in range(len(nft_markets)):
        nft_select = nft_markets[i]
        temp_dict = dict(
                         id = nft_select["id"],
                         asset_platform_id  = nft_select["asset_platform_id"],
                         floor_price_native = nft_select["floor_price"]["native_currency"],
                         floor_price_usd    = nft_select["floor_price"]["usd"],
                         market_cap_usd     = nft_select["market_cap"]["usd"],
                         volume_24h_usd     = nft_select["volume_24h"]["usd"]
                        )
        nft_all.append(temp_dict)
            
    return pd.DataFrame(nft_all)                     


In [21]:
df_nft_markets = get_nft_markets(10, "h24_volume_usd_desc")

In [22]:
df_nft_markets.id[7]

'azuki'

In [26]:
def get_nft_hist(nft_id, num_days):
    hist_params = {"days": num_days}
    hist_url = f"/nfts/{nft_id}/market_chart"
    
    nft_hist = get_response(hist_url, use_pro, hist_params, PRO_URL)
    all_time, all_floor_price_usd = [], []    

    floor_price_usd = nft_hist["floor_price_usd"]

    for i in range(len(floor_price_usd)):
        time = floor_price_usd[i][0]
        all_time.append(time)
        
        price = floor_price_usd[i][1]
        all_floor_price_usd.append(price)

    df_hist = pd.DataFrame(list(zip(all_time, all_floor_price_usd)),
                           columns =['Time', 'Price_usd'])

    df_hist["Time"] = pd.to_datetime(df_hist["Time"], unit = "ms")
    df_hist.set_index("Time", inplace = True)

    return df_hist    
    

In [45]:
df_hist = get_nft_hist(df_nft_markets.id[7], 180)

In [46]:
df_hist

,Price_usd
Time,
2023-05-10 00:05:18,28412.879393
2023-05-11 00:05:03,26798.381846
2023-05-12 00:05:02,26066.879501
2023-05-13 00:05:17,26064.795700
2023-05-14 00:05:19,24866.024982
...,...
2023-11-01 00:05:36,9501.750219
2023-11-02 00:05:42,9513.457400
2023-11-03 00:05:36,9277.535628


## Plot data

In [47]:
fig1 = df_hist.plot(title = f"Historical floor price for {df_nft_markets.id[7]}",
                    template = "seaborn",
                    labels = dict(index = "Time", value = "Price [USD]"))

In [48]:
fig1.show()